In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
import pandas as pd
import string
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from gensim.models import *
import gensim.downloader

import tensorflow as tf
from tensorflow.keras.utils import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
with open("/content/drive/MyDrive/DLNLP/rt-polarity.pos",encoding='latin-1') as f:
    pos = [line.rstrip() for line in f]
    
with open("/content/drive/MyDrive/DLNLP/rt-polarity.neg",encoding='latin-1') as f:
    neg = [line.rstrip() for line in f]
    


pos = pd.DataFrame(pos)
neg = pd.DataFrame(neg)

zeros = [0] * 5331
ones = [1] * 5331
pos = pd.concat((pos,pd.DataFrame(ones)),axis=1)
neg = pd.concat((neg,pd.DataFrame(zeros)),axis=1)

df_train = pd.concat((pos[0:4500],neg[0:4500]))
df_test = pd.concat((pos[4500:],neg[4500:]))

df_train.columns = ['text','class']
df_test.columns = ['text','class']

df_train['text'] = df_train.text.str.lower().str.replace(r'['+string.digits+string.punctuation+']', ' ')
df_train['text'] = df_train['text'].apply(lambda x: x.split())


In [ ]:
max = 0
txt = ''
for i in range(9000):
  if (len(df_train.iloc[i]['text']) >max):
    max = len(df_train.iloc[i]['text'])
    txt = df_train.iloc[i]['text']
              
print(txt,max)

['it', 'won', 't', 'harm', 'anyone', 'but', 'neither', 'can', 'i', 'think', 'of', 'a', 'very', 'good', 'reason', 'to', 'rush', 'right', 'out', 'and', 'see', 'it', 'after', 'all', 'it', 'll', 'probably', 'be', 'in', 'video', 'stores', 'by', 'christmas', 'and', 'it', 'might', 'just', 'be', 'better', 'suited', 'to', 'a', 'night', 'in', 'the', 'living', 'room', 'than', 'a', 'night', 'at', 'the', 'movies'] 53


In [ ]:
gv = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
w2v = gensim.downloader.load('word2vec-google-news-300') 

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
ft = gensim.downloader.load('fasttext-wiki-news-subwords-300') 

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
es = 300 ## embedding size

In [ ]:
str_len = 53
tz = Tokenizer(lower=True)
tz.fit_on_texts(df_train['text'])
X_train = tz.texts_to_sequences(df_train['text'])
X_train = pad_sequences(X_train, maxlen=str_len)
X_train = np.asarray(X_train)
print(X_train.shape)



(9000, 53)


In [ ]:
X_test = tz.texts_to_sequences(df_test['text'])
X_test = pad_sequences(X_test, maxlen=str_len)
X_test = np.asarray(X_test)
print(X_test.shape)

(1662, 53)


In [ ]:
       
gv_embedding = np.random.random((len(tz.word_index) + 1, es))
w2v_embedding = np.random.random((len(tz.word_index) + 1, es))
ft_embedding = np.random.random((len(tz.word_index) + 1, es))

for word,i in tz.word_index.items():
    try:
      gv_embedding[i] = gv.wv[word]
    except:
      gv_embedding[i] = np.zeros((es,))
    try:
      w2v_embedding[i] = w2v.wv[word]
    except:
      w2v_embedding[i] = np.zeros((es,))
    try:
      ft_embedding[i] = ft.wv[word]
    except:
      ft_embedding[i] = np.zeros((es,))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


In [ ]:
def ProjEmbLayer(embeddings, str_len):
    
    inputs = []
    output = []
    inp = Input(shape=(str_len,))
    for embedding in embeddings:  
        emb = Embedding(len(tz.word_index) + 1, es, weights=[embedding], trainable=False)(inp)
        emb = Reshape((-1,es,1))(emb)
        proj = projection(str_len)
        x = proj(Reshape((-1,es))(emb))
        output.append(Reshape((-1,es,1))(x))    
    return Model(inp, Concatenate(axis=-1)(output))


In [ ]:
def EmbLayer(embeddings, str_len):
    
    inputs = []
    output = []
    inp = Input(shape=(str_len,))
    for embedding in embeddings:  
        
        emb = Embedding(len(tz.word_index) + 1, es, weights=[embedding], trainable=False)(inp)
        emb = Reshape((-1,es,1))(emb)
        
        output.append(emb)    
    return Model(inp, Concatenate(axis=-1)(output))


In [ ]:

def projection(str_len):

    inp = Input(shape=(str_len, es))
    out = Dense(es,activation=None)(inp)

    return Model(inp, out)

def myDynamicMetaEmbedding(str_len):
    inp = Input(shape=(str_len, es, n))
    proj2mul  = Reshape((str_len, es,n))(inp)    ## (None, 30, 300, 3)
    proj = Permute((1,3, 2)) (proj2mul)          ## (None, 30, 3, 300)
    alphas = Dense(1,activation=None)(proj)      ## (None, 30, 3, 1)
    alphas = Activation('softmax')(alphas)       ## (None, 30, 3, 1)
    alphas2mul = Permute((1,3, 2)) (alphas)      ## (None, 30, 1, 3)
    x = multiply([proj2mul, alphas2mul])         ## (None, 30, 300, 3)
    out = Lambda(lambda t: K.sum(t, axis=-1))(x) ## (None, 30, 300)
    print('Out',out.shape)                      

    return Model(inp, out)

### WRONG ###

    # proj = Reshape((maxlen, es,n))(temp)
    # alphas = Dense(n,activation=None)(temp)
    # alphas = Activation('softmax')(alphas)
    # alphas = Reshape((maxlen,1,n))(alphas)
    # x = multiply([proj,alphas])
    # out = Lambda(lambda t: K.sum(t, axis=-1))(x)

    
    

In [ ]:
n=3
embLayer = ProjEmbLayer([w2v_embedding,gv_embedding, ft_embedding], str_len)
dme = myDynamicMetaEmbedding(str_len)
x = dme(embLayer.output)
x = LSTM(64)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(embLayer.input, out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, df_train['class'], batch_size=64, epochs=10, verbose=1, validation_split=0.1,callbacks=[callback])


print("TEST SET",model.evaluate(X_test, df_test['class']))

Out (None, 53, 300)
Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 53)]         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 53, 300)      5030400     input_18[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 53, 300)      5030400     input_18[0][0]                   
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 53, 300)      5030400     input_18[0][0]                   
_______________________________________________________________________

In [ ]:
n=3
embLayer = ProjEmbLayer([w2v_embedding,gv_embedding, ft_embedding], str_len)
dme = myDynamicMetaEmbedding(str_len)
x = dme(embLayer.output)
x = GRU(64)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(embLayer.input, out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit([X_train]*n, df_train['class'], batch_size=64, epochs=10, verbose=1, validation_split=0.1,callbacks=[callback])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

print("TEST SET",model.evaluate([X_test]*n, df_test['class']))

Out (None, 30, 300)
Model: "model_65"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_75 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 30, 300)      5843700     input_71[0][0]                   
_______________________________________________________________________

In [ ]:
n=1
embLayer = EmbLayer([w2v_embedding], str_len)
x = Reshape((str_len,es))(embLayer.output)
print(x.shape)
x = LSTM(128, dropout=0.2,return_sequences=True)(x)
x = LSTM(32)(x)
out = Dense(2, activation='sigmoid')(x)

w2v_model = Model(embLayer.input, out)
w2v_model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

w2v_model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
histor = w2v_model.fit(X_train, df_train['class'], batch_size=64, epochs=10, verbose=1, validation_split=0.1,callbacks=[callback])
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

print("TEST SET",w2v_model.evaluate(X_test, df_test['class']))

(None, 53, 300)
Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 53)]              0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 53, 300)           5030400   
_________________________________________________________________
reshape_47 (Reshape)         (None, 53, 300, 1)        0         
_________________________________________________________________
concatenate_8 (Concatenate)  (None, 53, 300, 1)        0         
_________________________________________________________________
reshape_48 (Reshape)         (None, 53, 300)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 53, 128)           219648    
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)   

In [ ]:
n=1
embLayer = EmbLayer([gv_embedding], str_len)
x = Reshape((str_len,es))(embLayer.output)
print(x.shape)
x = LSTM(128, dropout=0.2,return_sequences=True)(x)
x = LSTM(32)(x)
out = Dense(2, activation='sigmoid')(x)

gv_model = Model(embLayer.input, out)
gv_model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

gv_model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = gv_model.fit(X_train, df_train['class'], batch_size=64, epochs=10, verbose=1, validation_split=0.1,callbacks=[callback])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

print("TEST SET",gv_model.evaluate(X_test, df_test['class']))


(None, 53, 300)
Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 53)]              0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 53, 300)           5030400   
_________________________________________________________________
reshape_43 (Reshape)         (None, 53, 300, 1)        0         
_________________________________________________________________
concatenate_6 (Concatenate)  (None, 53, 300, 1)        0         
_________________________________________________________________
reshape_44 (Reshape)         (None, 53, 300)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 53, 128)           219648    
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)   

In [ ]:
n=1

embLayer = EmbLayer([ft_embedding], str_len)
x = Reshape((str_len,es))(embLayer.output)
print(x.shape)
x = LSTM(128, dropout=0.2,return_sequences=True)(x)
x = LSTM(32)(x)
out = Dense(2, activation='sigmoid')(x)

ft_model = Model(embLayer.input, out)
ft_model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

ft_model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = ft_model.fit(X_train, df_train['class'], batch_size=64, epochs=10, verbose=1, validation_split=0.1,callbacks=[callback])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

print("TEST SET",ft_model.evaluate(X_test, df_test['class']))

(None, 53, 300)
Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 53)]              0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 53, 300)           5030400   
_________________________________________________________________
reshape_49 (Reshape)         (None, 53, 300, 1)        0         
_________________________________________________________________
concatenate_9 (Concatenate)  (None, 53, 300, 1)        0         
_________________________________________________________________
reshape_50 (Reshape)         (None, 53, 300)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 53, 128)           219648    
_________________________________________________________________
lstm_14 (LSTM)               (None, 32)   